In [1]:
library(Rcrawler)
library(foreach)
library(doParallel)
library(scales)
library(qdapRegex)

Loading required package: iterators

Loading required package: parallel



In [2]:
Sys.setlocale("LC_CTYPE", locale="Chinese")

[1] "Chinese (Simplified)_China.936"

In [3]:
# parallel processing
cores=detectCores()
cl <- makeCluster(cores[1]-1) #not to overload your computer
registerDoParallel(cl)

In [4]:
# CSS patterns identified from website
pattern <- c(".content_table_R a", 
             ".content_table_R td:nth-child(4)",
             ".content_table_R td:nth-child(5)",
             ".content_table_R td:nth-child(6)",
             ".content_table_R td:nth-child(7)",
             ".content_table_R td:nth-child(8)")

# Names for each field collected
pattern_names <- c("id", 
                   "wins",
                   "battles",
                   "fleet",
                   "squad",
                   "activity")

scraper <- function(page){
  target_url <- paste("https://www.navyfield.com/Community/Ranking/List.aspx?page=",
                      toString(page),"&server=1&category=4&searchType=1&searchValue=", sep = "")
  web_content <- Rcrawler::ContentScraper(Url=target_url,
                                CssPatterns = pattern,
                                PatternsName = pattern_names, 
                                ManyPerPattern = TRUE, 
                                astext = TRUE, 
                                asDataFrame = FALSE)
  output <- as.data.frame(web_content)
  return(output)
}

rank_data <- foreach(i=1:500, .combine=rbind) %dopar% {
  tempMatrix = scraper(i) 
  tempMatrix #Equivalent to finalMatrix = rbind(finalMatrix, tempMatrix)
}

In [5]:
hof_pattern <- c(".rnum", 
             ".content_table_H td:nth-child(4)",
             ".content_table_H td:nth-child(5)",
             ".content_table_H td:nth-child(6)",
             ".content_table_H td:nth-child(7)")

hof_pattern_names <- c("id", 
                   "wins_total",
                   "fleet",
                   "squad",
                   "activity")

hof_scraper <- function(page){
    target_url <- paste("https://www.navyfield.com/Community/HallofFame/List.aspx?page=",
                        toString(page),"&server=&sort=1&searchType=1&searchValue=", sep = "")
    web_content <- Rcrawler::ContentScraper(Url=target_url,
                                  CssPatterns = hof_pattern,
                                  PatternsName = hof_pattern_names, 
                                  ManyPerPattern = TRUE, 
                                  astext = TRUE, 
                                  asDataFrame = FALSE)
    output <- as.data.frame(web_content)
    return(output)
}


hof_data <- foreach(i=1:7, .combine=rbind) %dopar% {
  tempMatrix = hof_scraper(i) 
  tempMatrix 
}

hof_data$activity = rm_between(hof_data$activity, '(', ')', extract=TRUE)
hof_data <- as.data.frame(lapply(hof_data, function(x) {
  gsub("\u00A0", "", x) 
})) 
hof_data$wins <- sub("/.*", "", hof_data$wins_total)
hof_data$battles <- sub(".*/", "", hof_data$wins_total)
hof_clean <- subset(hof_data, select = c(id, fleet:battles))

In [6]:
nf_data <- rbind(rank_data, hof_clean)
nf_data$id <- trimws(nf_data$id, which = c("both"))
nf_data$fleet <- trimws(nf_data$fleet, which = c("both"))
nf_data$squad <- trimws(nf_data$squad, which = c("both"))
nf_data$activity <- trimws(nf_data$activity, which = c("both"))
nf_data$activity <- gsub(" ", "", as.character(nf_data$activity))
nf_data$activity <- gsub("PST", "", as.character(nf_data$activity))
nf_data$wins <- as.numeric(gsub(",", "", as.character(nf_data$wins)))
nf_data$battles <- as.numeric(gsub(",", "", as.character(nf_data$battles)))
nf_data$win_rate <- nf_data$wins/nf_data$battles

In [17]:
# See results
substrRight <- function(x, n){
  as.numeric(substr(x, nchar(x)-n+1, nchar(x)))
}

substrLeft <- function(x, n){
  as.numeric(substr(x, 1, n))
}

fleet_power <- function(fleet_name, month, year){
    fleet_data <- subset(nf_data, fleet==fleet_name)
    attach(fleet_data)
    fleet_best <- fleet_data[order(-win_rate),]
    fleet_best$win_rate <- percent(fleet_best$win_rate, accuracy = 0.1)
    detach(fleet_data)
    return(head(subset(fleet_best,battles > 5000 & substrRight(fleet_best$activity,4)==year & substrLeft(fleet_best$activity,2)>month),20))
}

In [8]:
# NF players that win and/or disconnect most of the time (<_<)
# Only players active in last 3 months and have had 5000+ battles on record are listed
attach(nf_data)
nf_best <- nf_data[order(-win_rate),]
nf_best$win_rate <- percent(nf_best$win_rate, accuracy = 0.1)
detach(nf_data)
head(subset(nf_best,battles > 5000 & substrRight(nf_best$activity,4)==2021 & substrLeft(nf_best$activity,2)>4),20)

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1115,joooooo,6274,7852,REGALIA,-,06.27.2021,79.9%
937,nuke666,7065,9005,Russian Alliance,AborzSQ,07.04.2021,78.5%
1761,baozi77,4280,5546,-,-,07.06.2021,77.2%
1492,pr_ingress,4987,6466,TEAM PRO BOTS,-,07.06.2021,77.1%
244,jedizorro,13139,17241,Dango Daikazoku,-,07.07.2021,76.2%
10008,yyss2221,20495,27369,海风舰队,海風奔馳,07.06.2021,74.9%
898,betep,7238,9859,Russian Alliance,Nakhimov,06.21.2021,73.4%
246,k435345,13128,18023,-,-,07.04.2021,72.8%
243,greattak,13151,18241,-,-,07.06.2021,72.1%


In [33]:
# 深海舰队
shenhai <- fleet_power('深海舰队',4,2021)
shenhai

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1247,znfecu,5775,8102,深海舰队,勇敢天下,07.06.2021,71.3%
1286,b911753382a,5654,9676,深海舰队,-,07.07.2021,58.4%
1508,zcx19740112,4937,8454,深海舰队,深渊,05.06.2021,58.4%
2022,zzwotu,3711,6362,深海舰队,勇敢天下,07.06.2021,58.3%
554,cccwww,9353,16966,深海舰队,勇敢天下,07.07.2021,55.1%
1917,chalex001,3971,7228,深海舰队,-,07.06.2021,54.9%
1913,wjf197657,3978,7357,深海舰队,勇敢天下,06.25.2021,54.1%
1502,ctvcai007,4965,9211,深海舰队,勇敢天下,07.06.2021,53.9%
1257,erissoft,5748,10673,深海舰队,深渊,06.27.2021,53.9%


In [32]:
# 兔子舰队
rabbit <- fleet_power('兔子.CHN.',4,2021)
rabbit

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1130,la2gajaltmot,6220,9046,兔子.CHN.,World,07.07.2021,68.8%
146,tskspree,15629,22953,兔子.CHN.,World,07.06.2021,68.1%
2,laoji,37414,55153,兔子.CHN.,-,07.06.2021,67.8%
1626,hina272k,4610,6944,兔子.CHN.,Bravery,07.06.2021,66.4%
239,dreamcaca0,13290,20495,兔子.CHN.,World,07.06.2021,64.8%
433,zilong001,10621,17009,兔子.CHN.,pretender,07.04.2021,62.4%
337,aircraftcarrier,11668,18717,兔子.CHN.,Hermit,07.06.2021,62.3%
1217,liu001001,5858,9536,兔子.CHN.,讨债办,06.29.2021,61.4%
269,maykel,12709,20753,兔子.CHN.,World,05.20.2021,61.2%


In [31]:
# 荣誉舰队
rongyu <- fleet_power('新荣誉',4,2021)
rongyu

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1627,guxiaoqiang,4608,7816,新荣誉,虎贲军,07.06.2021,59.0%
2075,hsjj005,3581,6184,新荣誉,天下,05.29.2021,57.9%
1606,zeon0079,4658,8215,新荣誉,海狼团,07.06.2021,56.7%
2318,leviathan179,3085,5443,新荣誉,海狼团,07.05.2021,56.7%
2493,laowang888,2841,5072,新荣誉,海狼团,07.06.2021,56.0%
1960,zwl1972,3858,7025,新荣誉,-,07.06.2021,54.9%
2502,a1843452343,2829,5258,新荣誉,虎贲军,06.04.2021,53.8%
2598,qwe3147,2692,5014,新荣誉,海狼团,07.06.2021,53.7%
724,haksyx2003,8279,15463,新荣誉,海狼团,07.06.2021,53.5%


In [30]:
# 中华联合舰队
zhonglian <- fleet_power('中华联合舰队',4,2021)
zhonglian

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1215,liming,5873,9039,中华联合舰队,-,07.06.2021,65.0%
1240,nfship001,5805,9730,中华联合舰队,-,07.06.2021,59.7%
1831,ssdy2736,4129,6944,中华联合舰队,海魂,06.06.2021,59.5%
66,ak47ok,19993,34091,中华联合舰队,中华联合二队,07.06.2021,58.6%
326,hsx330136,11768,20123,中华联合舰队,-,07.06.2021,58.5%
450,daiyinjj,10428,17833,中华联合舰队,中华联合一队,07.06.2021,58.5%
1668,qq95568466,4507,7861,中华联合舰队,-,07.04.2021,57.3%
1326,lee1,5548,9678,中华联合舰队,-,07.04.2021,57.3%
2453,themyth,2887,5066,中华联合舰队,中华联合一队,06.24.2021,57.0%


In [27]:
# The Guardians Fleet
tgf <- fleet_power('Dango Daikazoku',4,2021)
tgf

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
244,jedizorro,13139,17241,Dango Daikazoku,-,07.07.2021,76.2%
649,hanyizhou,8624,13137,Dango Daikazoku,-,07.04.2021,65.6%
1812,kf222,4178,6881,Dango Daikazoku,Dragon,07.06.2021,60.7%
392,yixi,11126,18854,Dango Daikazoku,Dragon,07.05.2021,59.0%
1576,yupeng1982,4756,8137,Dango Daikazoku,Dragon,06.27.2021,58.4%
312,bibber,11994,20524,Dango Daikazoku,-,07.06.2021,58.4%
10030,iloveasu,7847,13464,Dango Daikazoku,-,07.07.2021,58.3%
413,ttlz,10912,18761,Dango Daikazoku,Dragon,07.02.2021,58.2%
411,skybear991,10925,18821,Dango Daikazoku,Dragon,07.03.2021,58.0%


In [29]:
# Team Pro Bots
tpb <- fleet_power('TEAM PRO BOTS',4,2021)
tpb

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1492,pr_ingress,4987,6466,TEAM PRO BOTS,-,07.06.2021,77.1%
951,vosu,6958,9696,TEAM PRO BOTS,TheBestOfBest,07.04.2021,71.8%
357,ded111,11469,16527,TEAM PRO BOTS,$CHMUTZ,07.07.2021,69.4%
548,portunus,9406,14009,TEAM PRO BOTS,$CHMUTZ,07.04.2021,67.1%
31,painlesskiss,24103,36764,TEAM PRO BOTS,-,07.04.2021,65.6%
778,schoko,7967,13126,TEAM PRO BOTS,$CHMUTZ,07.07.2021,60.7%
338,hollowman_eu,11666,20122,TEAM PRO BOTS,$CHMUTZ,07.06.2021,58.0%
1083,bestofchill,6398,11112,TEAM PRO BOTS,$CHMUTZ,07.04.2021,57.6%
44,aslaner,22533,39303,TEAM PRO BOTS,-,07.04.2021,57.3%


In [23]:
# BAMF Fleet
bamf <- fleet_power('BAMF FLEET',4,2021)
bamf

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1461,kenvahnz,5052,7277,BAMF FLEET,NERFED,07.06.2021,69.4%
696,ooh_kill_em,8420,12148,BAMF FLEET,Legacy,06.24.2021,69.3%
79,tappo01,19008,27986,BAMF FLEET,DooM,06.28.2021,67.9%
195,koll_,14378,21525,BAMF FLEET,Legacy,07.04.2021,66.8%
10097,lordfail,8977,13625,BAMF FLEET,DooM,06.25.2021,65.9%
336,saladrex,11670,18448,BAMF FLEET,NERFED,06.09.2021,63.3%
108,wilsonjordan,16874,26900,BAMF FLEET,#Karma,05.01.2021,62.7%
220,admiralkoth,13731,21950,BAMF FLEET,NERFED,05.15.2021,62.6%
33,huskie,23779,38290,BAMF FLEET,BULLS,07.06.2021,62.1%


In [22]:
# Russian Alliance
ra <- fleet_power('Russian Alliance',4,2021)
ra

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
937,nuke666,7065,9005,Russian Alliance,AborzSQ,07.04.2021,78.5%
898,betep,7238,9859,Russian Alliance,Nakhimov,06.21.2021,73.4%
1774,adorius,4256,6717,Russian Alliance,AborzSQ,05.15.2021,63.4%
465,osn3179,10258,16281,Russian Alliance,AborzSQ,07.06.2021,63.0%
816,xxvetalxx,7731,12338,Russian Alliance,Makarov,07.04.2021,62.7%
690,syava_mehan,8436,13676,Russian Alliance,-,07.04.2021,61.7%
1500,varangian,4968,8078,Russian Alliance,-,07.05.2021,61.5%
2175,gother_nfru,3354,5622,Russian Alliance,-,07.06.2021,59.7%
135,reisereise,15979,26909,Russian Alliance,AborzSQ,07.04.2021,59.4%


In [19]:
# 中国蓝舰队
zgl <- fleet_power('中国蓝舰队',4,2021)
zgl

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
736,wang82840506,8251,13612,中国蓝舰队,☆ ☆ ☆,07.05.2021,60.6%
1853,zjtz000000,4098,6775,中国蓝舰队,☆ ☆ ☆,07.07.2021,60.5%
1062,jovacn,6461,11275,中国蓝舰队,★ ☆ ★,07.06.2021,57.3%
1806,f9977,4190,7390,中国蓝舰队,★ ★ ★,05.28.2021,56.7%
378,wang811019,11249,19847,中国蓝舰队,-,06.10.2021,56.7%
242,kxjh,13166,23694,中国蓝舰队,★ ★ ☆,06.15.2021,55.6%
1943,e9977,3889,7023,中国蓝舰队,☆ ☆ ☆,05.22.2021,55.4%
134,hz111111,16041,29193,中国蓝舰队,☆ ☆ ☆,07.06.2021,54.9%
1411,solo3162,5230,9519,中国蓝舰队,★ ★ ★,07.06.2021,54.9%


In [20]:
# 北方舰队
bf <- fleet_power('北方舰队',4,2021)
bf

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1536,dongbeiwang,4857,7463,北方舰队,Odin,07.06.2021,65.1%
463,bin2014,10270,16585,北方舰队,THOR,07.05.2021,61.9%
504,bfhjzrb1319,9849,16422,北方舰队,-,07.06.2021,60.0%
2086,langhero,3561,6175,北方舰队,THOR,07.06.2021,57.7%
2049,andy_366,3650,6390,北方舰队,Odin,07.05.2021,57.1%
1518,gbsb8,4907,8616,北方舰队,北方天骄,06.29.2021,57.0%
992,abcdkent,6798,12247,北方舰队,THOR,07.06.2021,55.5%
1218,playab,5857,10611,北方舰队,-,06.11.2021,55.2%
2384,yaoyaopi,2994,5587,北方舰队,THOR,07.06.2021,53.6%


In [21]:
# Ostseepiraten Fleet
osp <- fleet_power('Ostseepiraten',4,2021)
osp

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
181,steuben_eu,14600,22575,Ostseepiraten,Stoertebeker,05.02.2021,64.7%
2195,ikken3,3319,5456,Ostseepiraten,P_Beneke,07.06.2021,60.8%
634,ikken4,8745,14913,Ostseepiraten,P_Beneke,07.06.2021,58.6%
10087,pamel,10252,17698,Ostseepiraten,-,07.06.2021,57.9%
163,bluestar_eu,15086,26202,Ostseepiraten,corsair,07.06.2021,57.6%
1309,kpt_jack,5586,9993,Ostseepiraten,Stoertebeker,07.01.2021,55.9%
1232,hippo83,5827,10463,Ostseepiraten,P_Beneke,05.23.2021,55.7%
1107,flausmaus,6311,11475,Ostseepiraten,Rum_u_Ehre,07.04.2021,55.0%
658,jolly_roger2,8576,15594,Ostseepiraten,Rum_u_Ehre,07.05.2021,55.0%


In [34]:
# REGALIA
rgla <- fleet_power('REGALIA',4,2021)
rgla

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1115,joooooo,6274,7852,REGALIA,-,06.27.2021,79.9%
2188,deredador,3327,5831,REGALIA,-,07.04.2021,57.1%
500,unosco,9864,18492,REGALIA,-,07.04.2021,53.3%
523,deseo5,9627,18212,REGALIA,-,07.05.2021,52.9%
1071,translei,6436,12435,REGALIA,-,06.27.2021,51.8%
1187,yoshyy,5970,11698,REGALIA,-,07.06.2021,51.0%
218,loquic12,13761,27072,REGALIA,Renovation,07.06.2021,50.8%
1332,shyk,5528,10896,REGALIA,-,06.27.2021,50.7%
921,oropeludo,7117,14192,REGALIA,-,06.28.2021,50.1%


In [35]:
# ARMADA CHILENA
ach <- fleet_power('ARMADA CHILENA',4,2021)
ach

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
178,makuvex,14663,27188,ARMADA CHILENA,-,06.13.2021,53.9%
382,cianuro51,11222,20977,ARMADA CHILENA,THE IMMORTALS,06.27.2021,53.5%
414,lmario666,10901,20789,ARMADA CHILENA,-,07.04.2021,52.4%
1763,carlosii,4273,8261,ARMADA CHILENA,"O""Higgins",07.03.2021,51.7%
1330,xeloborg89,5535,10792,ARMADA CHILENA,-,05.27.2021,51.3%
385,klython1,11196,21890,ARMADA CHILENA,-,06.27.2021,51.1%
2090,jotabe,3547,6985,ARMADA CHILENA,THE IMMORTALS,07.04.2021,50.8%
298,lalo_14,12222,24156,ARMADA CHILENA,THE IMMORTALS,07.06.2021,50.6%
152,alex_nilton,15525,30858,ARMADA CHILENA,THE IMMORTALS,07.04.2021,50.3%


In [36]:
# Flota Chile
fch <- fleet_power('Flota Chile',4,2021)
fch

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
187,cuadrad,14541,26107,Flota Chile,-,07.06.2021,55.7%
1204,kulebrakai,5907,10817,Flota Chile,-,06.27.2021,54.6%
374,xaloooo,11276,21167,Flota Chile,-,06.10.2021,53.3%
1102,gabriel_03,6332,11966,Flota Chile,Nanox,06.22.2021,52.9%
1106,chilotecocoa,6319,11944,Flota Chile,-,06.27.2021,52.9%
1594,breaka,4696,8891,Flota Chile,-,07.07.2021,52.8%
594,nitroxx,9053,17183,Flota Chile,ALDEA,06.27.2021,52.7%
764,elite_,8064,15397,Flota Chile,-,07.06.2021,52.4%
304,azsnar,12123,23150,Flota Chile,-,07.05.2021,52.4%


In [37]:
# Imperium
imp <- fleet_power('Imperium',4,2021)
imp

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
925,latinohot,7105,11405,Imperium,IMMORTALS,07.06.2021,62.3%
14,mariouchiha,28225,46011,Imperium,IMMORTALS,07.07.2021,61.3%
145,omegawhite,15699,25888,Imperium,IMMORTALS,05.12.2021,60.6%
107,nightmare29,17091,28868,Imperium,IMMORTALS,06.20.2021,59.2%
70,aildinchen,19550,33081,Imperium,Aggressor,07.04.2021,59.1%
589,delta56,9102,15716,Imperium,Veterans,07.07.2021,57.9%
661,ssepan,8569,14928,Imperium,-,07.07.2021,57.4%
1276,whitewolf200,5687,9910,Imperium,IMMORTALS,06.23.2021,57.4%
271,chefboy,12692,22296,Imperium,IMMORTALS,07.04.2021,56.9%


In [38]:
# Armored Angels
aa <- fleet_power('Armored Angels',4,2021)
aa

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
362,rainbow112,11450,18278,Armored Angels,Titans SQ,07.01.2021,62.6%
530,nager,9588,15876,Armored Angels,-,05.08.2021,60.4%
240,wishbonej,13230,23225,Armored Angels,-,05.14.2021,57.0%
173,will211,14762,27809,Armored Angels,-,07.03.2021,53.1%
200,hideship12,14235,26991,Armored Angels,Titans SQ,06.26.2021,52.7%
612,zemex,8921,17037,Armored Angels,Titans SQ,07.06.2021,52.4%
1058,wawa300,6467,12632,Armored Angels,Titans SQ,07.03.2021,51.2%
123,djr63,16381,32061,Armored Angels,Saintz SQ,07.04.2021,51.1%
49,mido,22060,43198,Armored Angels,Titans SQ,07.06.2021,51.1%


In [39]:
# HELL BOYS
hb <- fleet_power('HELL BOYS',4,2021)
hb

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1160,_vicon_,6080,9802,HELL BOYS,ZETAS,06.29.2021,62.0%
20,altsein,26005,44322,HELL BOYS,-,07.04.2021,58.7%
147,hitachi88,15602,26782,HELL BOYS,ZETAS,07.07.2021,58.3%
28,patatoo,24588,42484,HELL BOYS,-,06.13.2021,57.9%
199,det_ka,14239,24877,HELL BOYS,ZETAS,07.06.2021,57.2%
335,dakota775,11675,20512,HELL BOYS,ZETAS,06.06.2021,56.9%
522,andr2776,9656,17182,HELL BOYS,ZETAS,07.06.2021,56.2%
432,xelementx,10639,19198,HELL BOYS,-,07.06.2021,55.4%
1206,stranic,5900,10878,HELL BOYS,-,07.06.2021,54.2%


In [41]:
# Reiwa Fleet
rbgz <- fleet_power('Reiwa Fleet',4,2021)
rbgz

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
2206,y4427,3299,5019,Reiwa Fleet,-,07.06.2021,65.7%
2147,wanco2121,3403,5523,Reiwa Fleet,-,05.22.2021,61.6%
2262,aituki,3192,5185,Reiwa Fleet,-,07.03.2021,61.6%
104,zatti,17182,27968,Reiwa Fleet,-,06.08.2021,61.4%
1679,hesky,4486,7966,Reiwa Fleet,-,07.06.2021,56.3%
1066,sigemori,6448,12035,Reiwa Fleet,-,06.30.2021,53.6%
975,captainmaru,6864,13007,Reiwa Fleet,-,07.07.2021,52.8%
2637,cx817us,2639,5042,Reiwa Fleet,-,06.20.2021,52.3%
2170,wakasa3n,3368,6569,Reiwa Fleet,-,07.03.2021,51.3%


In [40]:
# Homeless
homeless <- fleet_power('-',4,2021)
homeless

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1761,baozi77,4280,5546,-,-,07.06.2021,77.2%
246,k435345,13128,18023,-,-,07.04.2021,72.8%
243,greattak,13151,18241,-,-,07.06.2021,72.1%
1167,magey2,6049,8957,-,-,05.10.2021,67.5%
936,fs4025paz,7072,10593,-,-,07.06.2021,66.8%
68,huitainan_a,19586,29367,-,-,07.06.2021,66.7%
591,laopu,9066,13637,-,-,07.07.2021,66.5%
333,berserker120,11714,18026,-,-,07.05.2021,65.0%
668,z11285208,8551,13310,-,-,07.06.2021,64.2%
